In [1]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from psd_tools import PSDImage
from tqdm import tqdm

In [2]:
def convert_psd_to_red_mask(psd_path, color_threshold=50):
    """
    PSDファイルを読み込み、赤色チャンネルをベースにした二値マスクを生成する
    
    Args:
        psd_path: PSDファイルのパス
        color_threshold: 赤色判定の閾値（デフォルト: 50）
        
    Returns:
        numpy.ndarray: 二値マスク画像（赤色部分が1、それ以外が0）
    """
    try:
        # PSDファイルを開く
        psd = PSDImage.open(psd_path)
        # PSDイメージを合成画像として取得
        composite_image = psd.composite()
        # numpy配列に変換
        img_array = np.array(composite_image)
        
        # RGBAの配列から赤色チャンネルを抽出
        if len(img_array.shape) == 3 and img_array.shape[2] == 4:  # RGBAの場合
            red_channel = img_array[:, :, 0]
            green_channel = img_array[:, :, 1]
            blue_channel = img_array[:, :, 2]
            alpha_channel = img_array[:, :, 3]
            
            # 赤色マスクの条件: 赤成分が高く、緑と青が低い、透明度がある
            red_mask = ((red_channel > green_channel + color_threshold) & 
                        (red_channel > blue_channel + color_threshold) & 
                        (alpha_channel > 0))
        elif len(img_array.shape) == 3 and img_array.shape[2] == 3:  # RGBの場合
            red_channel = img_array[:, :, 0]
            green_channel = img_array[:, :, 1]
            blue_channel = img_array[:, :, 2]
            
            # 赤色マスクの条件
            red_mask = ((red_channel > green_channel + color_threshold) & 
                        (red_channel > blue_channel + color_threshold))
        else:
            # グレースケールまたは対応していない形式
            print(f"警告: 対応していない画像形式です（shape: {img_array.shape}）")
            return None
        
        # 二値マスクに変換（True = 1, False = 0）
        binary_mask = red_mask.astype(np.uint8)
        
        return binary_mask
    except Exception as e:
        print(f"エラー: PSDファイル {psd_path} の処理中にエラーが発生しました: {str(e)}")
        return None

def convert_psd_folder_to_png(input_folder, output_folder, color_threshold=50, resize=None):
    """
    フォルダ内のすべてのPSDファイルをPNGマスクに変換する
    
    Args:
        input_folder: 入力PSDファイルのあるフォルダパス
        output_folder: 出力PNGファイルを保存するフォルダパス
        color_threshold: 赤色判定の閾値（デフォルト: 50）
        resize: リサイズする場合はタプル(width, height)で指定（デフォルト: None）
    """
    # 出力フォルダがなければ作成
    os.makedirs(output_folder, exist_ok=True)
    
    # 入力フォルダ内のPSDファイルを取得
    psd_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.psd')]
    
    if not psd_files:
        print(f"警告: {input_folder} にPSDファイルが見つかりませんでした。")
        return
    
    print(f"合計 {len(psd_files)} 個のPSDファイルを処理します...")
    
    # 処理カウンター
    success_count = 0
    error_count = 0
    
    # 各PSDファイルを処理
    for psd_file in tqdm(psd_files):
        psd_path = os.path.join(input_folder, psd_file)
        
        # PSDをマスクに変換
        mask_array = convert_psd_to_red_mask(psd_path, color_threshold)
        
        if mask_array is not None:
            # ファイル名（拡張子なし）を取得
            base_name = os.path.splitext(psd_file)[0]
            
            # マスクをPIL Imageに変換
            mask_img = Image.fromarray(mask_array)
            
            # リサイズが指定されている場合
            if resize is not None:
                mask_img = mask_img.resize(resize, Image.NEAREST)
            
            # PNG形式で保存
            output_path = os.path.join(output_folder, f"{base_name}.png")
            mask_img.save(output_path, format='PNG')
            
            # マスク内に何か検出されたか確認（デバッグ用）
            if np.any(mask_array > 0):
                detection_status = "マスク検出"
            else:
                detection_status = "マスクなし（空のマスク）"
            
            print(f"処理完了: {psd_file} → {base_name}.png ({detection_status})")
            success_count += 1
        else:
            error_count += 1
    
    # 結果表示
    print("\n処理結果:")
    print(f"成功: {success_count}, エラー: {error_count}")
    print(f"変換されたPNGファイルは {output_folder} に保存されました。")

# 使用例
# 以下の行のコメントを外して実行してください
input_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/labelsTs'  # PSDファイルがあるフォルダ
output_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/labelsTsROI'  # 出力先フォルダ
convert_psd_folder_to_png(input_folder, output_folder, color_threshold=50, resize=(512, 512))

合計 19 個のPSDファイルを処理します...


  0%|          | 0/19 [00:00<?, ?it/s]Unknown image resource 1092
/Users/yuma/Yuma-Kanematsu/nnUNet/.venv/lib/python3.10/site-packages/psd_tools/psd/tagged_blocks.py:264: UserWarning: Unknown key: b'CAI '
  warn(message)
Unknown key: b'CAI '
/Users/yuma/Yuma-Kanematsu/nnUNet/.venv/lib/python3.10/site-packages/psd_tools/psd/tagged_blocks.py:279: UserWarning: Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
  warn(message)
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
/Users/yuma/Yuma-Kanematsu/nnUNet/.venv/lib/python3.10/site-packages/psd_tools/psd/tagged_blocks.py:264: UserWarning: Unknown key: b'OCIO'
  warn(message)
Unknown key: b'OCIO'
/Users/yuma/Yuma-Kanematsu/nnUNet/.venv/lib/python3.10/site-packages/psd_tools/psd/tagged_blocks.py:279: UserWarning: Unknown tagged block: b'OCIO', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x1bdo ... =170'
  warn(message

処理完了: 6870292_23_R_FLOOR-3.psd → 6870292_23_R_FLOOR-3.png (マスク検出)
処理完了: 6870292_20_R_FLOOR-3.psd → 6870292_20_R_FLOOR-3.png (マスク検出)
処理完了: 6870292_19_R_FLOOR-3.psd → 6870292_19_R_FLOOR-3.png (マスク検出)
処理完了: 6887036_16_L_FLOOR-3.psd → 6887036_16_L_FLOOR-3.png (マスク検出)
処理完了: 6887036_12_L_FLOOR-2.psd → 6887036_12_L_FLOOR-2.png (マスク検出)
処理完了: 6955010_21_R_FLOOR-3.psd → 6955010_21_R_FLOOR-3.png (マスク検出)
処理完了: 6955010_18_R_FLOOR-2.psd → 6955010_18_R_FLOOR-2.png (マスク検出)
処理完了: 6955010_22_R_FLOOR-3.psd → 6955010_22_R_FLOOR-3.png (マスク検出)
処理完了: 6887036_15_L_FLOOR-3.psd → 6887036_15_L_FLOOR-3.png (マスク検出)


Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
Unknown key: b'OCIO'
Unknown tagged block: b'OCIO', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x1bdo ... =170'
Unknown key: b'GenI'
Unknown tagged block: b'GenI', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x0bge ... =84'
 58%|█████▊    | 11/19 [00:02<00:01,  7.55it/s]Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
Unknown key: b'OCIO'
Unknown tagged block: b'OCIO', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x1bdo ... =170'
Unknown key: b'GenI'
Unknown tagged block: b'GenI', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x0bge ... =84'
Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... 

処理完了: 6955010_23_R_FLOOR-3.psd → 6955010_23_R_FLOOR-3.png (マスク検出)
処理完了: 6887036_14_L_FLOOR-3.psd → 6887036_14_L_FLOOR-3.png (マスク検出)
処理完了: 6887036_17_L_FLOOR-2.psd → 6887036_17_L_FLOOR-2.png (マスク検出)
処理完了: 6887036_13_L_FLOOR-3.psd → 6887036_13_L_FLOOR-3.png (マスク検出)
処理完了: 6870292_16_R_FLOOR-2.psd → 6870292_16_R_FLOOR-2.png (マスク検出)
処理完了: 6955010_24_R_FLOOR-3.psd → 6955010_24_R_FLOOR-3.png (マスク検出)
処理完了: 6955010_19_R_FLOOR-3.psd → 6955010_19_R_FLOOR-3.png (マスク検出)
処理完了: 6955010_20_R_FLOOR-3.psd → 6955010_20_R_FLOOR-3.png (マスク検出)
処理完了: 6870292_21_R_FLOOR-3.psd → 6870292_21_R_FLOOR-3.png (マスク検出)


100%|██████████| 19/19 [00:02<00:00,  7.77it/s]

処理完了: 6870292_22_R_FLOOR-3.psd → 6870292_22_R_FLOOR-3.png (マスク検出)

処理結果:
成功: 19, エラー: 0
変換されたPNGファイルは /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/labelsTsROI に保存されました。


In [6]:
# 指定したフォルダ内の画像をすべてグレースケールに変換する関数
def convert_images_to_grayscale(input_folder, output_folder):
    """
    指定したフォルダ内の画像をすべてグレースケールに変換する
    
    Args:
        input_folder: 入力画像ファイルのあるフォルダパス
        output_folder: 出力グレースケール画像を保存するフォルダパス
    """
    # 出力フォルダがなければ作成
    os.makedirs(output_folder, exist_ok=True)
    
    # 入力フォルダ内の画像ファイルを取得
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print(f"警告: {input_folder} に画像ファイルが見つかりませんでした。")
        return
    
    print(f"合計 {len(image_files)} 個の画像ファイルを処理します...")
    
    # 処理カウンター
    success_count = 0
    error_count = 0
    
    # 各画像ファイルを処理
    for image_file in tqdm(image_files):
        image_path = os.path.join(input_folder, image_file)
        
        try:
            # 画像を開く
            img = Image.open(image_path)
            
            # グレースケールに変換
            gray_img = img.convert('L')
            
            # ファイル名（拡張子なし）を取得
            base_name = os.path.splitext(image_file)[0]
            
            # グレースケール画像を保存
            output_path = os.path.join(output_folder, f"{base_name}.png")
            gray_img.save(output_path, format='PNG')
            
            print(f"処理完了: {image_file} → {base_name}.png")
            success_count += 1
        except Exception as e:
            print(f"エラー: 画像 {image_file} の処理中にエラーが発生しました: {str(e)}")
            error_count += 1
    
    # 結果表示
    print("\n処理結果:")
    print(f"成功: {success_count}, エラー: {error_count}")
    print(f"変換されたグレースケール画像は {output_folder} に保存されました)。")

# 使用例
# 以下の行のコメントを外して実行してください
input_folder_gray = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/imagesTs'  # 入力画像フォルダ 
output_folder_gray = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/imagesTs_1'  # 出力先フォルダ
convert_images_to_grayscale(input_folder_gray, output_folder_gray)

合計 19 個の画像ファイルを処理します...


100%|██████████| 19/19 [00:00<00:00, 121.91it/s]

処理完了: 6870292_19_R_FLOOR-3_0000.png → 6870292_19_R_FLOOR-3_0000.png
処理完了: 6955010_18_R_FLOOR-2_0000.png → 6955010_18_R_FLOOR-2_0000.png
処理完了: 6887036_12_L_FLOOR-2_0000.png → 6887036_12_L_FLOOR-2_0000.png
処理完了: 6870292_21_R_FLOOR-3_0000.png → 6870292_21_R_FLOOR-3_0000.png
処理完了: 6955010_20_R_FLOOR-3_0000.png → 6955010_20_R_FLOOR-3_0000.png
処理完了: 6887036_14_L_FLOOR-3_0000.png → 6887036_14_L_FLOOR-3_0000.png
処理完了: 6887036_17_L_FLOOR-2_0000.png → 6887036_17_L_FLOOR-2_0000.png
処理完了: 6870292_22_R_FLOOR-3_0000.png → 6870292_22_R_FLOOR-3_0000.png
処理完了: 6955010_23_R_FLOOR-3_0000.png → 6955010_23_R_FLOOR-3_0000.png
処理完了: 6887036_16_L_FLOOR-3_0000.png → 6887036_16_L_FLOOR-3_0000.png
処理完了: 6955010_22_R_FLOOR-3_0000.png → 6955010_22_R_FLOOR-3_0000.png
処理完了: 6870292_23_R_FLOOR-3_0000.png → 6870292_23_R_FLOOR-3_0000.png
処理完了: 6955010_21_R_FLOOR-3_0000.png → 6955010_21_R_FLOOR-3_0000.png
処理完了: 6870292_20_R_FLOOR-3_0000.png → 6870292_20_R_FLOOR-3_0000.png
処理完了: 6887036_15_L_FLOOR-3_0000.png → 6887036_15